In [1]:
from keras.datasets import cifar10
(X_train,Y_train),(X_test,Y_test)=cifar10.load_data()

Using Theano backend.


# Tensorflow

In [2]:
print(X_train.shape)
print(X_test.shape)

(50000, 3, 32, 32)
(10000, 3, 32, 32)


In [3]:
print(X_train[0])

[[[ 59  43  50 ..., 158 152 148]
  [ 16   0  18 ..., 123 119 122]
  [ 25  16  49 ..., 118 120 109]
  ..., 
  [208 201 198 ..., 160  56  53]
  [180 173 186 ..., 184  97  83]
  [177 168 179 ..., 216 151 123]]

 [[ 62  46  48 ..., 132 125 124]
  [ 20   0   8 ...,  88  83  87]
  [ 24   7  27 ...,  84  84  73]
  ..., 
  [170 153 161 ..., 133  31  34]
  [139 123 144 ..., 148  62  53]
  [144 129 142 ..., 184 118  92]]

 [[ 63  45  43 ..., 108 102 103]
  [ 20   0   0 ...,  55  50  57]
  [ 21   0   8 ...,  50  50  42]
  ..., 
  [ 96  34  26 ...,  70   7  20]
  [ 96  42  30 ...,  94  34  34]
  [116  94  87 ..., 140  84  72]]]


In [4]:
X_train=X_train/255.0
X_test=X_test/255.0

In [5]:
X_train[0]

array([[[ 0.23137255,  0.16862745,  0.19607843, ...,  0.61960784,
          0.59607843,  0.58039216],
        [ 0.0627451 ,  0.        ,  0.07058824, ...,  0.48235294,
          0.46666667,  0.47843137],
        [ 0.09803922,  0.0627451 ,  0.19215686, ...,  0.4627451 ,
          0.47058824,  0.42745098],
        ..., 
        [ 0.81568627,  0.78823529,  0.77647059, ...,  0.62745098,
          0.21960784,  0.20784314],
        [ 0.70588235,  0.67843137,  0.72941176, ...,  0.72156863,
          0.38039216,  0.3254902 ],
        [ 0.69411765,  0.65882353,  0.70196078, ...,  0.84705882,
          0.59215686,  0.48235294]],

       [[ 0.24313725,  0.18039216,  0.18823529, ...,  0.51764706,
          0.49019608,  0.48627451],
        [ 0.07843137,  0.        ,  0.03137255, ...,  0.34509804,
          0.3254902 ,  0.34117647],
        [ 0.09411765,  0.02745098,  0.10588235, ...,  0.32941176,
          0.32941176,  0.28627451],
        ..., 
        [ 0.66666667,  0.6       ,  0.63137255, ...,

In [6]:
Y_train[0]

array([6], dtype=uint8)

In [7]:
# create a one hot vector for label
from keras.utils import np_utils

Y_train=np_utils.to_categorical(Y_train)
Y_test=np_utils.to_categorical(Y_test)

In [8]:
Y_train[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.])

# Tensorflow

In [9]:
import tensorflow as tf
X = tf.placeholder("float",[None,32,32,3]) # as our dataset
Y = tf.placeholder("float",[None,10])

In [10]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

W_C1 = init_weights([3,3,3,32])# 3x3x3 conv, 32 outputs
W_C2 = init_weights([3,3,32,64])# 3x3x32 conv, 64 outputs
W_C3 = init_weights([3, 3, 64, 128])# 3x3x64 conv, 128 outputs

W_FC = init_weights([128 * 4 * 4, 625]) # FC 128 * 4 * 4 inputs, 625 outputs
W_O = init_weights([625, 10])         # FC 625 inputs, 10 outputs (labels)

p_keep_conv = tf.placeholder("float") #for dropouts as percentage
p_keep_hidden = tf.placeholder("float")


In [11]:
def model(X, W_C1, W_C2, W_C3, W_FC, W_O, p_keep_conv,p_keep_hidden):
    
    C1 = tf.nn.relu(tf.nn.conv2d(X,W_C1,
                                strides=[1,1,1,1], padding = "SAME")) 
    
    P1 = tf.nn.max_pool(C1,ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding = "SAME" ) # 1st pooling layer shape =(?,14,14,32)
    
    D1 = tf.nn.dropout(P1,p_keep_conv) # 1st dropout at conv
    
    C2 = tf.nn.relu(tf.nn.conv2d(D1,W_C2,
                                strides=[1,1,1,1], padding = "SAME")) # 2nd convoultion layer shape=(?, 14, 14, 62)
    
    P2 = tf.nn.max_pool(C2,ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding = "SAME" ) # 2nd pooling layer shape =(?,7,7,64)
    
    D2 = tf.nn.dropout(P2,p_keep_conv) # 2nd dropout at conv
    
    
    C3 = tf.nn.relu(tf.nn.conv2d(D2,W_C3,
                                strides=[1,1,1,1], padding = "SAME")) # 3rd convoultion layer shape=(?, 7, 7, 128)
    
    P3 = tf.nn.max_pool(C3,ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding = "SAME" ) # 3rd pooling layer shape =(?,4,4,128)
    
    P3 = tf.reshape(P3, [-1, W_FC.get_shape().as_list()[0]])    # reshape to (?, 2048)
    D3 = tf.nn.dropout(P3, p_keep_conv) # 3rd dropout at conv
    
    FC = tf.nn.relu(tf.matmul(D3,W_FC))
    FC = tf.nn.dropout(FC, p_keep_hidden) #droput at fc
    
    output = tf.matmul(FC,W_O)
    
    return output

In [12]:
Y_pred = model(X, W_C1, W_C2, W_C3, W_FC, W_O, p_keep_conv,p_keep_hidden)

In [13]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Y_pred ,labels = Y))
 # compute mean cross entropy (softmax is applied internally)

In [14]:
optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)

In [15]:
predict_op = tf.argmax(Y_pred, 1) # at predict time, evaluate the argmax of the logistic regression

In [16]:
#reshape as images according to tf 
X_train = X_train.reshape(-1,32,32,3) 
X_test = X_test.reshape(-1,32,32,3)

In [17]:
epochs = 50
import numpy as np
with tf.Session() as sess:
    # you need to initialize all variables
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        
        for start, end in zip(range(0, len(X_train), 128), range(128, len(X_train)+1, 128)): 
            sess.run(optimizer,feed_dict={X : X_train[start:end] , Y : Y_train[start:end],
                                          p_keep_conv: 0.8, p_keep_hidden: 0.5})
        if epoch % 10 == 0:
            accuracy= np.mean(np.argmax(Y_test, axis=1) == 
                              sess.run(predict_op, feed_dict={X: X_test, p_keep_conv: 1.0, p_keep_hidden: 1.0}))
                              
            print("epoch : {} and accuracy : {}" .format(epoch, accuracy))
            
            print("testing labels for test data")
            print(sess.run(predict_op, feed_dict={X: X_test,p_keep_conv: 1.0, p_keep_hidden: 1.0}))
            
    print("Final accuracy : {}" .format(np.mean(np.argmax(Y_test, axis=1) ==
                         sess.run(predict_op, feed_dict={X: X_test, p_keep_conv: 1.0, p_keep_hidden: 1.0}))))

epoch : 0 and accuracy : 0.2622
testing labels for test data
[3 8 8 ..., 3 1 7]
epoch : 10 and accuracy : 0.5798
testing labels for test data
[3 8 8 ..., 5 1 7]
epoch : 20 and accuracy : 0.6304
testing labels for test data
[3 8 8 ..., 5 1 7]
epoch : 30 and accuracy : 0.6497
testing labels for test data
[6 8 8 ..., 5 1 7]
epoch : 40 and accuracy : 0.6497
testing labels for test data
[3 8 8 ..., 5 1 7]
Final accuracy : 0.6332
